In [1]:
import pandas as pd
import sys


In [2]:
#Save data to dataframes
df_train = pd.read_csv("./part2/hepatitis-training", delimiter=r"\s+");
df_test = pd.read_csv("./part2/hepatitis-test", delimiter=r"\s+");

In [3]:
class Node:
    def __init__(self, attribute, left_node = None, right_node = None, is_leaf = False):
        self.attribute = attribute
        self.left_node = left_node
        self.right_node = right_node 
        self.is_leaf = is_leaf
    

In [4]:
def build_tree(instances, attributes):
    if instances.empty:
        return
    elif calc_impurity():
        return
    elif attributes.empty:
        return
    else:

        best_impurity = sys.maxsize
        best_att = None
        best_true = None
        best_false = None


        for attribute in attributes:

                if attribute == "Class":
                    continue

                true_instance = instances.loc[instances[attribute] == True]
                false_instance = instances.loc[instances[attribute] == False]

                true_impurity = calc_impurity(true_instance, len(instances))
                false_impurity = calc_impurity(false_instance, len(instances))
                
                true_weighted = len(true_instance) / len(instances) * true_impurity
                false_weighted = len(false_instance) / len(instances) * false_impurity

                if(true_weighted + false_weighted < best_impurity):
                    best_impurity = true_weighted + false_weighted

                    best_att = attribute
                    best_true = true_instance
                    best_false = false_instance

        left = build_tree(best_true, attributes.remove(best_att))
        right = build_tree(best_false, attributes.remove(best_att))

        return Node(best_att, left, right)
    



In [5]:
def calc_impurity(instances, total):

    try:
        die = instances.Class.value_counts().die
    except:
        die = 0

    try:
        live = instances.Class.value_counts().live
    except:
        live = 0

    live = instances.Class.value_counts().live

    return die / total * live / total

In [6]:
root_node = build_tree(df_train, df_train.columns)

TypeError: calc_impurity() missing 2 required positional arguments: 'instances' and 'total'